In [1]:
import io
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack
from scipy import ndimage
from sklearn import svm, metrics

from sklearn.metrics import classification_report, confusion_matrix
import imageio
import matplotlib.image as mpimage
import cv2
import glob
import h5py
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import datetime as dt
from six.moves import range

In [2]:
train_path="/home/csed/ELC_activity/train"


In [4]:
train_labels=os.listdir(train_path)

train_labels.sort()

print(train_labels)
nb_classes= 5
global_features_train=[]
train_classes=[]



i,j=0,0
k=0

['ada', 'haha', 'idi', 'sasa', 'uda']


In [7]:
from matplotlib.pyplot import imread
for training_name in train_labels:
    path=os.path.join(train_path,training_name,'*')
    files=glob.glob(path)
    current_label=training_name
    k=1
    for fl in files:
        image=imread(fl)
        global_feature=np.hstack([image])
        train_classes.append(current_label)
        global_features_train.append(global_feature)

        i+=1
        k+1
    print("[status] processed folder: {}".format(current_label))
    j+=1
print("[status] completed global feature extraction..")
print("[status] feature vector size {}".format(np.array(global_features_train).shape))
print("[status] training labels {}".format(np.array(train_classes).shape))


[status] processed folder: ada
[status] processed folder: haha
[status] processed folder: idi
[status] processed folder: sasa
[status] processed folder: uda
[status] completed global feature extraction..
[status] feature vector size (493, 32, 32, 4)
[status] training labels (493,)


In [8]:
#labels = (np.arange(nb_classes) == labels[:,None]).astype(np.float32)
targetNames=np.unique(train_classes)
le=LabelEncoder()
target=le.fit_transform(train_classes)
print("[status] training labels encoded...")

[status] training labels encoded...


In [9]:
n_samples, nx, ny,ch=np.array(global_features_train).shape
d2_global_features=np.array(global_features_train).reshape((n_samples, nx*ny*ch))
#scaler=MinMaxScaler(feature_range=(0,1))
#rescaled_features=scaler.fit_transform(d2_global_features)

In [10]:
print("[Status] feature vector normalized...")

print("[Status] target label{}".format(target))
print("[Status] target label shape {}".format(target.shape))


[Status] feature vector normalized...
[Status] target label[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 

In [11]:
h5f_data=h5py.File('/home/csed/ELC_activity/output_train_svm/dataelc.h5','w')
h5f_data.create_dataset('dataset_1',data=np.array(d2_global_features))

h5f_label=h5py.File('/home/csed/ELC_activity/output_train_svm/labelselc.h5','w')
h5f_label.create_dataset('dataset_1',data=np.array(target))


<HDF5 dataset "dataset_1": shape (493,), type "<i8">

In [12]:
h5f_data.close()
h5f_label.close()

In [13]:
h5f_data=h5py.File('/home/csed/ELC_activity/output_train_svm/dataelc.h5','r')
h5f_label=h5py.File('/home/csed/ELC_activity/output_train_svm/labelselc.h5','r')


In [14]:
global_features_string_train= h5f_data['dataset_1']
global_labels_string_train=h5f_label['dataset_1']
global_features_train=np.array(global_features_string_train)
global_labels_train=np.array(global_labels_string_train)


In [15]:
h5f_data.close()
h5f_label.close()

In [16]:
print("[Status] training feature shape: {}".format(global_features_train.shape))

print("[Status] labels shape: {}".format(global_labels_train.shape))

[Status] training feature shape: (493, 4096)
[Status] labels shape: (493,)


In [17]:
clf = svm.SVC(kernel='linear')

In [18]:
from sklearn.model_selection import KFold

In [19]:
cv = KFold(n_splits=4, random_state=1, shuffle=True)

In [20]:
scores = cross_val_score(clf,global_features_train,global_labels_train , scoring='accuracy', cv=cv, n_jobs=-1)

In [21]:
from numpy import mean
from numpy import std
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.992 (0.006)


In [22]:
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf, global_features_train,global_labels_train , cv=10)
conf_mat = confusion_matrix(global_labels_train, y_pred)


In [23]:
print("Confusion matrix:\n%s" % conf_mat)

Confusion matrix:
[[100   0   0   0   0]
 [  0 100   0   0   0]
 [  1   0  99   0   0]
 [  0   0   0  93   0]
 [  2   0   0   0  98]]


In [24]:
cv = KFold(n_splits=2, random_state=1, shuffle=True)
scores = cross_val_score(clf,global_features_train,global_labels_train , scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))
y_pred = cross_val_predict(clf, global_features_train,global_labels_train , cv=10)
conf_mat = confusion_matrix(global_labels_train, y_pred)
print("Confusion matrix:\n%s" % conf_mat)


Accuracy: 0.988 (0.004)
Confusion matrix:
[[100   0   0   0   0]
 [  0 100   0   0   0]
 [  1   0  99   0   0]
 [  0   0   0  93   0]
 [  2   0   0   0  98]]
